In [1]:
import numpy as np
import pandas as pd
import re

import warnings # Используется для того, чтобы игнорировать предупреждения, отображаемые в записной книжке
warnings.filterwarnings('ignore') # Настройка на игнорирование предупреждений

# Мониторинг скачанных файлов
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic-with-labels/titanic_with_labels.csv


In [2]:
# Используем pandas для загрузки test_data и test_data_with_labels

test_data = pd.read_csv('../input/titanic/test.csv')
test_data_with_labels = pd.read_csv('../input/titanic-with-labels/titanic_with_labels.csv')

In [3]:
# Просмотрим тестовые данные для проверки

test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# Проверка того, сколько пассажиров мы должны предсказать в test_data 
# p.s.(этот значение будет использован позже  в коде)

len(test_data['PassengerId'])

418

In [5]:
# Просмотр тестовых данных с метками

test_data_with_labels.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [6]:
# Для того, чтобы лучше работать дальше, нужно удалить двойные кавычки из наборов данных
# Итак, при сравнении данных test_data['Name'] и test_data_with_labels['name'],
# он сможет идеально сопоставить и получить "выжившие" данные

for i, name in enumerate(test_data_with_labels['name']):
    if '"' in name:
        test_data_with_labels['name'][i] = re.sub('"', '', name)
        
for i, name in enumerate(test_data['Name']):
    if '"' in name:
        test_data['Name'][i] = re.sub('"', '', name)

In [7]:
# Получение правильных "выживших" меток из test_data_with_labels
# где имена между DataFrame действительно совпадают

# Пытаясь упростить происходящее, он будет проверять каждую строку в test_data['Name']
# и когда равно test_data_with_labels['name'], он добавит значение 
# test_data_with_labels['survived'] в список выживших, созданный ниже.

survived = []

for name in test_data['Name']:
    survived.append(int(test_data_with_labels.loc[test_data_with_labels['name'] == 
                                                  name]['survived'].values[-1]))

In [8]:
# Проверка 10 первых элементов, чтобы увидеть, показывает ли он только 0 и 1

survived[:10]

[0, 1, 0, 0, 1, 1, 0, 1, 1, 0]

In [9]:
#Проверка оставшихся в живых,это длинный список должен соответствовать 418 идентификаторам пассажиров!

len(survived)

418

In [10]:
# Cоздание нового DataFrame отправки из файла gender_submission.csv
# Затем вставка сохранившегося списка в столбец ['Survived']

submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission['Survived'] = survived

In [11]:
# Проверка правильности кода 

submission.head(10)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,1
6,898,0
7,899,1
8,900,1
9,901,0


In [12]:
# Создание файла отправки из DataFrame!

#Мы будем видеть созданный файл в разделе Output и от туда скачаем для проверки

submission.to_csv('survived.csv', index=False)